# Формат Yolov8
Yolov8 хочет иметь следующий формат: папки train, val и test. Для каждой из них есть папка images и папка labels. Для каждого изображения в папке images есть файл .txt с тем же именем и соответствующими метками. Чтобы получить метки для сегментации, посмотрите JSON2YOLO на github <https://github.com/ultralytics/JSON2YOLO>.

Для индивидуального использования измените пути соответствующим образом

In [ ]:
# Получить имена всех файлов

import os
image_directory = 'data/images'
 
# получить имена всех файлов изображений
filenames = []
for filename in os.listdir(image_directory):
    filenames.append(filename[:-4]) # last 4 characters are '.jpg'

In [ ]:
# Первоначально данные об игле манометра и лицевой стороне манометра разделены. С помощью этого мы объединили метки.
# 0 соответствует торцу манометра, 1 - игле манометра

def merge_labels(gauge_face_path, gauge_needle_path, dst_path):
    for filename in filenames:
        with open(dst_path + filename+'.txt', 'w') as outfile:
            with open(gauge_face_path+filename+'.txt') as infile:
                outfile.write(infile.read())
            with open(gauge_needle_path+filename+'.txt') as infile:
                for line in infile:
                    line = '1' + line[1:]
                    outfile.write(line)

In [ ]:
gauge_face_path = 'data/gauge_face/annotations/bbox_labels_yolo/'
gauge_needle_path = 'data/gauge_needle/annotations/bbox_labels_yolo/'
path_bbox_labels = 'data/bbox_labels/'
merge_labels(gauge_face_path, gauge_needle_path, path_bbox_labels)
gauge_face_path = 'data/gauge_face/annotations/segmentation_labels_yolo/'
gauge_needle_path = 'data/gauge_needle/annotations/segmentation_labels_yolo/'
path_segmentation_labels = 'data/segmentation_labels/'
merge_labels(gauge_face_path, gauge_needle_path, path_segmentation_labels)


In [ ]:
# разделите изображения на обучающий, контрольный и тестовый наборы

import random

def split_dataset(filenames):
    # Предположим, что у вас есть список имен файлов с именем «all_filenames».
    random.shuffle(filenames)

    # вычислите размер каждого набора
    num_files = len(filenames)
    num_train = int(0.8 * num_files)  # 80% for training
    num_val = int(0.1 * num_files)   # 10% for validation
    num_test = num_files - num_train - num_val  # remaining 10% for test

    # Разделите список на три группы
    train_filenames = filenames[:num_train]
    val_filenames = filenames[num_train:num_train+num_val]
    test_filenames = filenames[num_train+num_val:]

    # Напечатайте размеры каждого набора
    print(f"Количество файлов в обучающем наборе: {len(train_filenames)}")
    print(f"Количество файлов в наборе для проверки: {len(val_filenames)}")
    print(f"Количество файлов в тестовом наборе: {len(test_filenames)}")
    
    return train_filenames, val_filenames, test_filenames


train_filenames, val_filenames, test_filenames = split_dataset(filenames)

In [ ]:
# Создание структуры папок
import os
dir_base = ['segmentation','detection']
modes = ['train', 'val', 'test']
for base in dir_base:
    for mode in modes:
        path = 'data/' + base + '/' + mode +'/images'
        os.makedirs(path, exist_ok=True)
        path = 'data/' + base + '/' + mode +'/labels'
        os.makedirs(path, exist_ok=True)

In [ ]:
import shutil

# скопировать изображение и файл метки с заданным именем файла в соответствующие папки в новой структуре папок
def copy_pair(src_dir, target_dir, file_name, mode):
    src = image_directory + '/' + file_name + ".jpg"
    dst = target_dir + '/' + mode +'/images/' + file_name + ".jpg"
    shutil.copy2(src, dst)
    src = src_dir + '/' + file_name + ".txt"
    dst = target_dir + '/' + mode +'/labels/' + file_name + ".txt"
    shutil.copy2(src, dst)

# для каждого набора скопируйте все метки и изображения из этого набора в соответствующие 
def copy_split(src_dir, target_dir):
    for name in train_filenames:
        copy_pair(src_dir, target_dir, name, 'train')
    for name in val_filenames:
        copy_pair(src_dir, target_dir, name, 'val')
    for name in test_filenames:
        copy_pair(src_dir, target_dir, name, 'test')

copy_split(path_segmentation_labels, 'data/segmentation')
copy_split(path_bbox_labels, 'data/detection')
